<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/339_WDO_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Workforce Development Orchestrator — State & Configuration Design

This section defines the **contract** that governs how the Workforce Development Orchestrator thinks, reasons, and produces outcomes. Rather than hiding logic inside opaque model calls, the design makes every major decision point explicit, inspectable, and configurable.

At a high level, this code does two things:

1. Defines a **transparent, auditable state object** that all agent nodes read from and write to
2. Defines a **CEO-friendly configuration layer** that controls thresholds, priorities, and behavior without changing code

Together, these form the backbone of a rules-driven, executive-ready AI system.

---

## 1. State as a First-Class Design Artifact

The `WorkforceDevelopmentOrchestratorState` is not just a data container — it is the **shared memory and decision ledger** for the entire agent.

Every node in the orchestrator:

* Reads from this state
* Adds structured results back into it
* Leaves an auditable trail of how conclusions were reached

This ensures the system remains **deterministic, explainable, and testable**.

### Why This Matters

Instead of letting decisions emerge implicitly from a single model call, the agent:

* Separates *data ingestion*, *analysis*, *prioritization*, and *reporting*
* Preserves intermediate reasoning artifacts
* Makes it possible to inspect, replay, or override decisions later

This is exactly what leaders expect from systems that influence workforce strategy.

---

## 2. Clear Separation of Responsibilities in State Design

The state is intentionally structured into logical layers that mirror how organizations reason about workforce change.

### 📥 Input & Planning

Fields like `employee_id`, `goal`, and `plan` establish **intent** up front.
This allows the agent to operate in:

* Individual mode (one employee)
* Role-level mode
* Organization-wide mode

The planning fields are template-driven in the MVP, reinforcing predictability and control.

---

### 📊 Data Ingestion (Single Source of Truth)

The state explicitly holds:

* Employees
* Roles
* Tasks
* Skills
* Automation signals
* Skill gaps
* Learning paths
* Role evolution scenarios

Each structure is documented inline, making the data contract self-describing. This avoids the common failure mode where agents silently assume schema details that later change.

---

### ⚡ Performance-Oriented Lookups

Lookup dictionaries like:

* `roles_lookup`
* `tasks_lookup`
* `employees_by_role`

exist purely to make the agent **fast and deterministic**.
This is a deliberate choice: reasoning speed and clarity are prioritized over recomputation or hidden caching.

---

## 3. Analysis Outputs Are Explicit, Not Implied

Rather than jumping straight to recommendations, the agent produces **named analytical artifacts**:

### Automation Risk Analysis

Captures:

* Which tasks are high / medium / low risk
* Aggregate risk scores
* Which employees are affected
* Why those risks exist

This makes automation impact something leaders can *inspect*, not just accept.

---

### Skill Gap Analysis

Skill gaps are enhanced with:

* Role context
* Automation risk context
* Urgency scores
* Learning path mappings

This transforms raw gaps into **actionable workforce insights**, not abstract deficiencies.

---

### Learning & Role Evolution Recommendations

Recommendations are not vague suggestions — they include:

* Match scores
* Readiness indicators
* Implementation priorities

This supports ROI-driven decisions such as:

> “Which investments reduce risk fastest?”

---

## 4. Built-In Prioritization and Metrics

The state includes **explicit prioritization outputs** and **summary metrics**, such as:

* Employees at risk
* High-priority skill gaps
* Roles requiring evolution
* Overall workforce readiness score

This allows the agent to produce:

* Executive summaries
* Dashboards
* Trend tracking over time

All without additional interpretation layers.

---

## 5. Errors and Metadata Are Treated Seriously

The `errors` field is additive and node-agnostic, allowing:

* Multiple nodes to report issues safely
* Partial execution without silent failure
* Clear post-mortems when something goes wrong

This is critical for trust in long-running or production agents.

---

## 6. Configuration as a Control Plane, Not a Footnote

The `WorkforceDevelopmentOrchestratorConfig` class is where **business control lives**.

Key design choices here are intentional:

### 🎯 Explicit Risk Thresholds

Automation risk thresholds are:

* Numeric
* Named
* Adjustable

This means leadership can change what “high risk” means **without retraining models or rewriting logic**.

---

### ⚖️ Weighted Skill Gap Prioritization

Skill gap urgency is driven by configurable weights:

* Automation exposure
* Role requirements
* Future relevance
* Performance impact

This makes prioritization **policy-driven**, not model-driven.

---

### 📚 Learning Recommendation Controls

Matching thresholds and top-N limits ensure:

* Recommendations stay focused
* Noise is controlled
* Outputs remain actionable

---

### 🔌 Modular Toolshed Integration

Feature flags like:

* `enable_prioritization`
* `enable_reporting`

allow capabilities to be turned on or off cleanly — a hallmark of mature system design.

---

## Why This Design Builds Executive Confidence

This code establishes a system where:

* Decisions are traceable
* Logic is configurable
* Outputs are measurable
* AI augments judgment instead of replacing it

Most importantly, it ensures the orchestrator can always answer:

> **“Why did the system recommend this?”**

That’s the difference between an AI demo and an AI system leaders are willing to trust.




# Workforce Development Orchestrator Agent

In [ ]:
# ============================================================================
# Workforce Development Orchestrator Agent
# ============================================================================

class WorkforceDevelopmentOrchestratorState(TypedDict, total=False):
    """State for Workforce Development Orchestrator Agent"""

    # Input fields
    employee_id: Optional[str]              # Specific employee to analyze (None = analyze all)

    # Goal & Planning fields (MVP: Fixed goal, template-based plan)
    goal: Dict[str, Any]                     # Goal definition (from goal_node)
    plan: List[Dict[str, Any]]              # Execution plan (from planning_node)

    # Data Ingestion
    employees: List[Dict[str, Any]]         # Loaded employee data
    # Structure per employee:
    # {
    #   "employee_id": "E001",
    #   "name": "Sarah Chen",
    #   "role_id": "R001",
    #   "department_id": "D001",
    #   "tenure_months": 24,
    #   "current_skills": ["communication", "data_entry", "process_design"],
    #   "performance_rating": 4.2
    # }

    roles: List[Dict[str, Any]]             # Loaded role data
    # Structure per role:
    # {
    #   "role_id": "R001",
    #   "role_name": "HR Coordinator",
    #   "department_id": "D001",
    #   "required_skills": ["communication", "data_entry", "process_design"],
    #   "future_skills": ["ai_tools", "automation_workflows"],
    #   "tasks": ["T001", "T002", "T003"]
    # }

    tasks: List[Dict[str, Any]]            # Loaded task data
    # Structure per task:
    # {
    #   "task_id": "T001",
    #   "task_name": "Maintain employee records",
    #   "role_id": "R001",
    #   "frequency": "weekly",
    #   "automation_risk_score": 0.70,
    #   "automation_risk_level": "high",
    #   "human_judgment_level": "low",
    #   "required_skills": ["data_entry", "process_design"],
    #   "risk_factors": ["structured_data", "repetitive_pattern"]
    # }

    skills: List[Dict[str, Any]]           # Loaded skill data
    # Structure per skill:
    # {
    #   "skill_id": "data_entry",
    #   "skill_name": "Data Entry",
    #   "skill_type": "technical",
    #   "category": "operations",
    #   "description": "Accurately input structured data into systems"
    # }

    automation_signals: List[Dict[str, Any]]  # Loaded automation signal data
    # Structure per signal:
    # {
    #   "signal_id": "AS001",
    #   "task_id": "T001",
    #   "signal_type": "structured_data",
    #   "severity": "high",
    #   "description": "Task relies on structured, predictable data fields"
    # }

    skill_gaps: List[Dict[str, Any]]       # Loaded skill gap data
    # Structure per gap:
    # {
    #   "gap_id": "SG001",
    #   "employee_id": "E001",
    #   "skill_id": "ai_tools",
    #   "gap_type": "missing_future_skill",
    #   "priority": "high",
    #   "role_requirement": true
    # }

    learning_paths: List[Dict[str, Any]]   # Loaded learning path data
    # Structure per path:
    # {
    #   "learning_path_id": "LP001",
    #   "path_name": "AI Tools for Knowledge Workers",
    #   "target_skill": "ai_tools",
    #   "courses": [...],
    #   "estimated_completion_weeks": 3,
    #   "difficulty_level": "beginner"
    # }

    role_evolution: List[Dict[str, Any]]    # Loaded role evolution data
    # Structure per evolution:
    # {
    #   "evolution_id": "RE001",
    #   "role_id": "R001",
    #   "current_role_name": "HR Coordinator",
    #   "evolution_type": "augmented",
    #   "description": "...",
    #   "automated_tasks": ["T001"],
    #   "augmented_tasks": ["T002"],
    #   "new_tasks": [...],
    #   "new_skills_required": ["ai_tools", "ai_collaboration"]
    # }

    # Data Lookups (for fast access)
    roles_lookup: Dict[str, Dict[str, Any]]  # role_id -> role dict
    tasks_lookup: Dict[str, Dict[str, Any]]  # task_id -> task dict
    skills_lookup: Dict[str, Dict[str, Any]]  # skill_id -> skill dict
    employees_lookup: Dict[str, Dict[str, Any]]  # employee_id -> employee dict
    tasks_by_role: Dict[str, List[Dict[str, Any]]]  # role_id -> list of tasks
    employees_by_role: Dict[str, List[Dict[str, Any]]]  # role_id -> list of employees

    # Automation Risk Analysis
    automation_risk_analysis: List[Dict[str, Any]]  # Analysis results per employee/role
    # Structure per analysis:
    # {
    #   "employee_id": Optional[str],  # None if role-level analysis
    #   "role_id": "R001",
    #   "high_risk_tasks": ["T001", "T002"],
    #   "medium_risk_tasks": ["T003"],
    #   "low_risk_tasks": ["T004"],
    #   "overall_risk_score": 0.65,
    #   "risk_level": "medium",
    #   "affected_employees": ["E001", "E002"],
    #   "recommendations": [...]
    # }

    # Skill Gap Analysis
    skill_gap_analysis: List[Dict[str, Any]]  # Enhanced gap analysis with context
    # Structure per gap:
    # {
    #   "gap_id": "SG001",
    #   "employee_id": "E001",
    #   "employee_name": "Sarah Chen",
    #   "role_id": "R001",
    #   "role_name": "HR Coordinator",
    #   "skill_id": "ai_tools",
    #   "skill_name": "AI Tools",
    #   "gap_type": "missing_future_skill",
    #   "priority": "high",
    #   "role_requirement": true,
    #   "automation_risk_context": "high",  # Employee's role has high automation risk
    #   "urgency_score": 0.85,
    #   "recommended_learning_path": "LP001"
    # }

    # Learning Path Recommendations
    learning_path_recommendations: List[Dict[str, Any]]  # Personalized recommendations
    # Structure per recommendation:
    # {
    #   "employee_id": "E001",
    #   "employee_name": "Sarah Chen",
    #   "learning_path_id": "LP001",
    #   "learning_path_name": "AI Tools for Knowledge Workers",
    #   "target_skill": "ai_tools",
    #   "match_score": 0.95,
    #   "priority": "high",
    #   "estimated_completion_weeks": 3,
    #   "prerequisites_met": true,
    #   "rationale": "Employee missing future skill required for role"
    # }

    # Role Evolution Recommendations
    role_evolution_recommendations: List[Dict[str, Any]]  # Role transformation suggestions
    # Structure per recommendation:
    # {
    #   "role_id": "R001",
    #   "role_name": "HR Coordinator",
    #   "evolution_id": "RE001",
    #   "evolution_type": "augmented",
    #   "description": "...",
    #   "affected_employees": ["E001", "E002"],
    #   "automated_tasks": ["T001"],
    #   "new_skills_required": ["ai_tools", "ai_collaboration"],
    #   "readiness_score": 0.75,
    #   "implementation_priority": "high"
    # }

    # Prioritization
    prioritized_gaps: List[Dict[str, Any]]  # Skill gaps sorted by priority
    prioritized_recommendations: List[Dict[str, Any]]  # Learning paths sorted by priority
    prioritized_evolutions: List[Dict[str, Any]]  # Role evolutions sorted by priority

    # Summary Metrics
    workforce_summary: Dict[str, Any]
    # Structure:
    # {
    #   "total_employees": 10,
    #   "total_roles": 5,
    #   "total_tasks": 15,
    #   "employees_at_risk": 6,  # Employees with high automation risk tasks
    #   "total_skill_gaps": 6,
    #   "high_priority_gaps": 3,
    #   "total_learning_recommendations": 6,
    #   "roles_requiring_evolution": 5,
    #   "overall_workforce_readiness_score": 0.65
    # }

    # Output
    workforce_report: str                   # Final markdown report
    report_file_path: Optional[str]         # Path to saved report file

    # Metadata
    errors: Annotated[List[str], operator.add]  # Any errors encountered (can be updated by multiple nodes)
    processing_time: Optional[float]        # Time taken to process


@dataclass
class WorkforceDevelopmentOrchestratorConfig:
    """Configuration for Workforce Development Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/workforce_development_reports"  # Where to save reports

    # Data file paths
    data_dir: str = "agents/data"
    employees_file: str = "employees.json"
    roles_file: str = "roles.json"
    tasks_file: str = "tasks.json"
    skills_file: str = "skills.json"
    automation_signals_file: str = "automation_signals.json"
    skill_gaps_file: str = "skills_gaps.json"
    learning_paths_file: str = "learning_paths.json"
    role_evolution_file: str = "role_evolution.json"

    # Automation Risk Thresholds (CEO-Friendly Transparency)
    automation_risk_thresholds: Dict[str, float] = field(default_factory=lambda: {
        "high": 0.7,      # >= 0.7 is high risk
        "medium": 0.4,    # 0.4-0.7 is medium risk
        "low": 0.0        # < 0.4 is low risk
    })

    # Skill Gap Priority Weights
    gap_priority_weights: Dict[str, float] = field(default_factory=lambda: {
        "automation_risk": 0.35,      # High automation risk = high priority
        "role_requirement": 0.30,      # Required for role = high priority
        "future_skill": 0.20,         # Future skill = medium priority
        "employee_performance": 0.15  # Performance impact = lower priority
    })

    # Learning Path Matching
    learning_path_match_threshold: float = 0.6  # Minimum match score for recommendation

    # Top N Recommendations
    top_n_gaps: int = 10
    top_n_learning_paths: int = 5
    top_n_role_evolutions: int = 5

    # Toolshed Integration
    enable_progress_tracking: bool = True   # Use toolshed.progress
    enable_prioritization: bool = True     # Use toolshed.prioritization
    enable_reporting: bool = True          # Use toolshed.reporting

